## CSV generation from Result txts (Accuracy summary)

In [ ]:
inputs = ['../results/csv_based/',
           '../results/csv_without_interms/']

ind = 0

root = inputs[ind]

output_root = root + 'SlidingWindowParameters/'

methods = ['ResNet', 'CNN', 'CNN+LSTM',
           'LSTM', 'convAuto', 'DFT', 'DCT']

import glob

In [ ]:
import numpy as np
import csv
import os.path
from os import path

def getAccuracies(filename):
    f = open(filename, "r")
    accuracies = []
    acc_full = 0.0
    acc_fea = 0.0
    for x in f:
        if 'Accuracy:' in x:
            accuracies.append(np.float32(x[10:]))
        if 'Acc_full' in x:
            acc_full = np.float32(x[10:])
        if 'Acc_fea' in x:
            if 'Acc_features' in x:
                acc_fea = np.float32(x[14:])
            else:
                acc_fea = np.float32(x[10:])
    return accuracies, acc_full, acc_fea

def parseFileName(filename, root, method):
    num = len(root) + len(method) + 1
    f = filename[num:]
    attempt = f[:1]
    skill = f[2:]
    if skill[0] == 'K':
        skill = skill[:4]
    elif skill[0] == 'S':
        skill = skill[:6]
    elif skill[0] == 'N':
        skill = skill[:6]
    win = f[3+len(skill):]
    if win[0] == '1':
        win = win[:3]
    else:
        win = win[:2]
    step = f[4+len(skill)+len(win):]
    step = step[:2]
    return attempt,skill,win,step,method

def CSVify(a, acc_full, acc_fea, skill, win_size, step_size, method,attempt):
    filename = output_root + str(win_size) + '_' + str(step_size) + '.csv'
    exists = path.exists(filename)
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = [ 'method', 'skill', 'run',
                      'Trial_1_Model', 'Trial_2_Model',
                      'Trial_3_Model', 'Trial_4_Model', 'Trial_5_Model',
                      'Trial_1_SVM', 'Trial_2_SVM', 'Trial_3_SVM',
                      'Trial_4_SVM', 'Trial_5_SVM',
                      'Acc_Full', 'Acc_Fea']
        writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
        if not exists:
            writer.writeheader()
            print('Header written for ' + filename)
        if len(a) == 10:
            #print('Both Model and SVM acurracies')
            temp = {'method': method, 'skill': skill, 'run':attempt,
                    'Trial_1_Model':a[0], 'Trial_1_SVM':a[1],
                    'Trial_2_Model':a[2], 'Trial_2_SVM':a[3],
                    'Trial_3_Model':a[4], 'Trial_3_SVM':a[5],
                    'Trial_4_Model':a[6], 'Trial_4_SVM':a[7],
                    'Trial_5_Model':a[8], 'Trial_5_SVM':a[9],
                    'Acc_Full':acc_full, 'Acc_Fea':acc_fea
                       }
            writer.writerow(temp)
        else:
            #print('Only SVM accuracies')
            temp = {'method': method, 'skill': skill, 'run':attempt,
                    'Trial_1_Model':0.0, 'Trial_1_SVM':a[0],
                    'Trial_2_Model':0.0, 'Trial_2_SVM':a[1],
                    'Trial_3_Model':0.0, 'Trial_3_SVM':a[2],
                    'Trial_4_Model':0.0, 'Trial_4_SVM':a[3],
                    'Trial_5_Model':0.0, 'Trial_5_SVM':a[4],
                    'Acc_Full':acc_full, 'Acc_Fea':acc_fea
                       }
            writer.writerow(temp)

In [ ]:
csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)
print('Removing old csvs')
for c in csvs:
    os.remove(c)
    print('Deleted ' + c)

csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)
print('Generating up-to-date csvs...')

for m in methods:
    txts = glob.glob(root + m + '/*.txt')
    txts.sort()
    for t in txts:
        accuracies, acc_full, acc_fea = getAccuracies(t)
        if len(accuracies) in [5, 10]:
            attempt,skill, win, step, method = parseFileName(t,root,m)
            CSVify(accuracies, acc_full, acc_fea, skill, win, step, m, attempt)
        else:
            print(t)

csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)

## Confusion Matrix processing

In [ ]:
inputs = ['../results/csv_based/',
           '../results/csv_without_interms/']

ind = 0

withoutInterms = (ind == 1)

root = inputs[ind]

output_root = root + 'ConfusionMatrices/'

methods = ['ResNet', 'CNN', 'CNN+LSTM',
           'LSTM', 'convAuto', 'DFT', 'DCT']

import glob

In [ ]:
import numpy as np
import csv
import os.path
from os import path

def getCMinfo(filename):
    f = open(filename, "r")
    #matrices = []
    model_matrix = []
    svm_matrix = []
    Model = True
    SVM = True
    for x in f:
        num_list = []
        if 'Full' in x:
            Model = True
            SVM = False
        if 'Fea' in x:
            Model = False
            SVM = True
        if '[' in x:
            line = x[1:]
            if line[0] == '[':
                line = line[1:]
            i = 0
            while i < len(line) and line[i] != ']':
                while i < len(line) and (line[i] == ' '):
                    i += 1
                while i < len(line) and line[i] != ' ' and line[i] != ']':
                    i += 1
                num = line[:i]
                line = line[i:]
                i = 0
                num_list.append(int(num))
            if Model:
                model_matrix.append(num_list)
            if SVM:
                svm_matrix.append(num_list)
            #print(num_list)
        #matrices.append(matrix)
    #print(matrix)
    return model_matrix, svm_matrix

def parseFileName(filename, root, method):
    num = len(root) + len(method) + 1
    f = filename[num:]
    attempt = f[:1]
    skill = f[2:]
    if skill[0] == 'K':
        skill = skill[:4]
    elif skill[0] == 'S':
        skill = skill[:6]
    elif skill[0] == 'N':
        skill = skill[:6]
    win = f[3+len(skill):]
    if win[0] == '1':
        win = win[:3]
    else:
        win = win[:2]
    step = f[4+len(skill)+len(win):]
    step = step[:2]
    return attempt,skill,win,step,method

def saveCMinfo(model_matrix, svm_matrix, method, win, step,attempt,skill):   
    filename = output_root + method + '_ConfusionMatrix.csv'
    exists = path.exists(filename)
    with open(filename, 'a', newline='') as csvfile:
        if withoutInterms:
            fieldnames = [  #For 2 classes: Novice and Expert
                           'window_size', 'step_size', 'run', 'skill', 'type',
                           'Novice_True_Positive', 'Novice_False_Positive',
                           'Expert_True_Positive', 'Expert_False_Positive',
                         ]
        else:
            fieldnames = [
                            #For 3 classes: Novice, Intermediate and Expert
                            'window_size', 'step_size', 'run', 'skill', 'type',
                            'Novice_True_Positive', 'Interm_insteadOf_Novice', 'Expert_insteadOf_Novice',
                            'Novice_insteadOf_Interm', 'Interm_True_Positive', 'Expert_insteadOf_Interm',
                            'Novice_insteadOf_Expert', 'Interm_insteadOf_Expert', 'Expert_True_Positive'
                         ]
        writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
        if not exists:
            writer.writeheader()
        if withoutInterms:
            i = 0
            while i < (model_matrix.shape[0]-1):
                    temp = {
                        'window_size':win, 'step_size':step, 'run':attempt, 'skill':skill, 'type':'Model',
                        'Novice_True_Positive':model_matrix[i][0], 'Expert_False_Positive':model_matrix[i][1],
                        'Novice_False_Positive':model_matrix[i+1][0], 'Expert_True_Positive':model_matrix[i+1][1]
                    }
                    writer.writerow(temp)
                    i += 2
            i = 0
            while i < (svm_matrix.shape[0]-1):
                    temp = {
                        'window_size':win, 'step_size':step, 'run':attempt, 'skill':skill, 'type':'SVM',
                        'Novice_True_Positive':svm_matrix[i][0], 'Expert_False_Positive':svm_matrix[i][1],
                        'Novice_False_Positive':svm_matrix[i+1][0], 'Expert_True_Positive':svm_matrix[i+1][1]
                    }
                    writer.writerow(temp)
                    i += 2
        else:
            i = 0
            while i < (model_matrix.shape[0]-2):
                    temp = {
                        'window_size':win, 'step_size':step,'run':attempt, 'skill':skill,'type':'Model',
                        'Novice_True_Positive':model_matrix[i][0],
                        'Interm_insteadOf_Novice':model_matrix[i][1],
                        'Expert_insteadOf_Novice':model_matrix[i][2],
                        
                        'Novice_insteadOf_Interm':model_matrix[i+1][0],
                        'Interm_True_Positive':model_matrix[i+1][1],
                        'Expert_insteadOf_Interm':model_matrix[i+1][2],
                        
                        'Novice_insteadOf_Expert':model_matrix[i+2][0],
                        'Interm_insteadOf_Expert':model_matrix[i+2][1],
                        'Expert_True_Positive':model_matrix[i+2][2]
                    }
                    writer.writerow(temp)
                    i += 3
            i = 0
            while i < (svm_matrix.shape[0]-2):
                    temp = {
                        'window_size':win, 'step_size':step,'run':attempt,'skill':skill,'type':'SVM',
                        'Novice_True_Positive':svm_matrix[i][0],
                        'Interm_insteadOf_Novice':svm_matrix[i][1],
                        'Expert_insteadOf_Novice':svm_matrix[i][2],
                        
                        'Novice_insteadOf_Interm':svm_matrix[i+1][0],
                        'Interm_True_Positive':svm_matrix[i+1][1],
                        'Expert_insteadOf_Interm':svm_matrix[i+1][2],
                        
                        'Novice_insteadOf_Expert':svm_matrix[i+2][0],
                        'Interm_insteadOf_Expert':svm_matrix[i+2][1],
                        'Expert_True_Positive':svm_matrix[i+2][2]
                    }
                    writer.writerow(temp)
                    i += 3

In [ ]:
csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)
print('Removing old csvs')
for c in csvs:
    os.remove(c)
    print('Deleted ' + c)

csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)
print('Generating up-to-date csvs...')

for m in methods:
    txts = glob.glob(root + m + '/*.txt')
    txts.sort()
    for t in txts:
        #print(m)
        if withoutInterms:
            model_matrix, svm_matrix = getCMinfo(t)
            #print(' ')
            attempt,skill, win, step, method = parseFileName(t,root,m)
            saveCMinfo(np.asarray(model_matrix), np.asarray(svm_matrix), method, win,step,attempt,skill)
        else:
            attempt,skill, win, step, method = parseFileName(t,root,m)
            #ResNet 150x60, CNN 150x60, CNN+LSTM 180x30, LSTM 180x30, ConvaAuto 120x60
            dontSkip = (m == 'ResNet' and int(win) == 150 and int(step) == 60)
            dontSkip = dontSkip or (m == 'CNN' and int(win) == 150 and int(step) == 60)
            dontSkip = dontSkip or (m == 'CNN+LSTM' and int(win) == 180 and int(step) == 30)
            dontSkip = dontSkip or (m == 'LSTM' and int(win) == 180 and int(step) == 30)
            dontSkip = dontSkip or (m == 'convAuto' and int(win) == 120 and int(step) == 60)
            if dontSkip:
                model_matrix, svm_matrix = getCMinfo(t)
                saveCMinfo(np.asarray(model_matrix), np.asarray(svm_matrix), method, win,step,attempt,skill)

csvs = glob.glob(output_root + '*.csv')
print(str(len(csvs)) + ' csv files in ' + output_root)